In [ ]:
%load_ext dotenv

%dotenv ../tests/.env.test
%dotenv -o ../.env

In [ ]:
from redbox.app import Redbox
from redbox.models.chain import RedboxQuery, RedboxState, ChainChatMessage

In [ ]:
app = Redbox(debug=False)

In [ ]:
s = RedboxState(
    request=RedboxQuery(
        question="@gadget what is it about?",
        s3_keys=[],
        user_uuid="5210311c-df38-430f-8451-df5fa28f0be3",  # Get this from the URL in the admin console while viewing your local superuser
        chat_history=[
            ChainChatMessage(role="user", text="What is this report?"),
            ChainChatMessage(
                role="ai", text="The report is Tony_Blair_Institute__Governing_in_the_Age_of_AI__May_2024"
            ),
        ],
        permitted_s3_keys=["ai_tests/Tony_Blair_Institute__Governing_in_the_Age_of_AI__May_2024__2_.pdf"],
    )
)

In [ ]:
response_text = ""
citations = []
activity_log = []


async def token_handler(s):
    global response_text
    response_text += s


async def citations_handler(cs):
    citations.extend(cs)


async def activity_handler(e):
    activity_log.append(e)


r = await app.run(
    input=s,
    response_tokens_callback=token_handler,
    citations_callback=citations_handler,
    activity_event_callback=activity_handler,
)

print("*" * 10)
print()
print(f"User: {r.request.question}")
print()
print("Activity Log:")
for log in activity_log:
    print(f"\t - {log.message}")
print()
print(f"AI: {response_text}")
print()
print("Citations: ")
for c in citations:
    print(c.text_in_answer)
    for source in c.sources:
        print(f"[{source.source_type}] {source.source}")
        print("\t" + source.highlighted_text_in_source)

In [ ]:
str(r.messages[-1])